In [2]:
import numpy as np
import torch

Training Data

In [3]:
# temperature, rainfall, humidity
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [4]:
# apples, oranges
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [5]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Linear Regression Model from Scratch

In [6]:
# Weights and biases

w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[ 1.0883, -0.5828,  1.9659],
        [ 0.7675, -0.4780, -1.0068]], requires_grad=True)
tensor([-0.9248,  0.6222], requires_grad=True)


Our model definition

In [7]:
def model(x):
    return x @ w.t() + b

## .t() provides the transpor & '@' is for matrix multiplication

In [8]:
# generating predictions
preds = model(inputs)
print(preds)

tensor([[124.0029, -18.6741],
        [172.6356, -36.0415],
        [129.6761, -55.0602],
        [157.7572,  21.0959],
        [155.8250, -62.7908]], grad_fn=<AddBackward0>)


In [9]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Loss Function

In [10]:
## Mean Squared Error
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(12966.6904, grad_fn=<DivBackward0>)


Computing Gradients

In [12]:
# Compute gradients
loss.backward()

In [13]:
print(w)
print(w.grad)

tensor([[ 1.0883, -0.5828,  1.9659],
        [ 0.7675, -0.4780, -1.0068]], requires_grad=True)
tensor([[  6344.8071,   4991.8975,   3625.7571],
        [ -9894.2021, -12267.3359,  -7360.9888]])


We need to reset the gradients back to zero to avoid unexpected results as it keeps adding the previous ones

In [14]:
w.grad.zero_()
b.grad.zero_()

print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [15]:
w, b

(tensor([[ 1.0883, -0.5828,  1.9659],
         [ 0.7675, -0.4780, -1.0068]], requires_grad=True),
 tensor([-0.9248,  0.6222], requires_grad=True))

Adjusting weights & biases using gradient descent

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights & biases
4. Adjust weights by subtracting a small quantity proportional to the gradient
5. Reset gradients to zero


In [16]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[124.0029, -18.6741],
        [172.6356, -36.0415],
        [129.6761, -55.0602],
        [157.7572,  21.0959],
        [155.8250, -62.7908]], grad_fn=<AddBackward0>)


In [17]:
# calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(12966.6904, grad_fn=<DivBackward0>)


In [18]:
# compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[  6344.8071,   4991.8975,   3625.7571],
        [ -9894.2021, -12267.3359,  -7360.9888]])
tensor([  71.7794, -122.2941])


In [19]:
# adjusting weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

1. We use torch.no_grad to indicate to Pytorch that we shouldn't track, calculate or modify gradients while updating weights & biases
2. we multiply gradients with a really small number, to ensure we don't modify weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient
3. after updating weights, gradients are reset to zero to avoid affecting any future computations

In [20]:
print(w)
print(b)

tensor([[ 1.0249, -0.6328,  1.9296],
        [ 0.8664, -0.3554, -0.9332]], requires_grad=True)
tensor([-0.9256,  0.6235], requires_grad=True)


In [21]:
# calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(9496.2373, grad_fn=<DivBackward0>)


Training for multiple epochs

In [22]:
# train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [23]:
# calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(774.2385, grad_fn=<DivBackward0>)


In [24]:
preds

tensor([[ 64.1087,  78.9495],
        [ 96.9183,  96.9244],
        [ 74.0691, 127.5888],
        [ 63.3559,  85.6610],
        [102.6594,  84.3569]], grad_fn=<AddBackward0>)

In [25]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

LINEAR REGRESSION USING PYTORCH BUILT-INs

In [26]:
import torch.nn as nn

In [27]:
# input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96, 70], [73, 67, 43],
                   [91, 88, 64], [87, 134, 58], [102, 43, 37],
                   [69, 96, 70], [73, 67, 43], [91, 88, 64],
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]],
                  dtype='float32')
# targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133],
                    [22, 37], [103, 119], [56, 70],
                    [81, 101], [119, 133], [22, 37],
                    [103, 119], [56, 70], [81, 101],
                    [119, 133], [22, 37], [103, 119]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

Dataset & DataLoader

In [28]:
from torch.utils.data import TensorDataset

In [29]:
# defining dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [30]:
from torch.utils.data import DataLoader

In [31]:
# define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [32]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.]])
tensor([[ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [103., 119.]])


nn.Linear

In [33]:
# define the model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.4563, -0.4000,  0.2973],
        [ 0.0457,  0.3154,  0.2735]], requires_grad=True)
Parameter containing:
tensor([-0.1772, -0.0130], requires_grad=True)


In [34]:
# parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4563, -0.4000,  0.2973],
         [ 0.0457,  0.3154,  0.2735]], requires_grad=True),
 Parameter containing:
 tensor([-0.1772, -0.0130], requires_grad=True)]

In [35]:
# generate predictions
preds = model(inputs)
preds

tensor([[19.1156, 36.2100],
        [25.1720, 49.3975],
        [ 3.1612, 62.0810],
        [40.1659, 28.3254],
        [13.7165, 52.5561],
        [19.1156, 36.2100],
        [25.1720, 49.3975],
        [ 3.1612, 62.0810],
        [40.1659, 28.3254],
        [13.7165, 52.5561],
        [19.1156, 36.2100],
        [25.1720, 49.3975],
        [ 3.1612, 62.0810],
        [40.1659, 28.3254],
        [13.7165, 52.5561]], grad_fn=<AddmmBackward0>)

Loss Function

In [36]:
import torch.nn.functional as F

In [37]:
# define loss function
loss_fn = F.mse_loss

In [38]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(3952.1506, grad_fn=<MseLossBackward0>)


Optimizer

We use stochastic gradient descent because samples are selected in batches instead of a single group

In [39]:
# define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Train the model

In [40]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        
        # training with batches of data
        for xb,yb, in train_dl:
            
            # generating predictions
            pred = model(xb)
            
            # calculating loss
            loss = loss_fn(pred, yb)
            
            # computing gradients
            loss.backward()
            
            # updating parameters using gradients
            opt.step()
            
            # resetting gradients to zero
            opt.zero_grad()
            
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [41]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 586.2150
Epoch [20/100], Loss: 151.1566
Epoch [30/100], Loss: 237.5135
Epoch [40/100], Loss: 189.8727
Epoch [50/100], Loss: 114.2773
Epoch [60/100], Loss: 91.0531
Epoch [70/100], Loss: 58.3859
Epoch [80/100], Loss: 41.2041
Epoch [90/100], Loss: 33.5619
Epoch [100/100], Loss: 34.5074


In [42]:
preds = model(inputs)
preds

tensor([[ 58.7322,  71.2756],
        [ 82.8354,  98.6887],
        [113.7203, 135.6432],
        [ 31.1343,  42.5925],
        [ 97.1858, 112.4529],
        [ 58.7322,  71.2756],
        [ 82.8354,  98.6887],
        [113.7203, 135.6432],
        [ 31.1343,  42.5925],
        [ 97.1858, 112.4529],
        [ 58.7322,  71.2756],
        [ 82.8354,  98.6887],
        [113.7203, 135.6432],
        [ 31.1343,  42.5925],
        [ 97.1858, 112.4529]], grad_fn=<AddmmBackward0>)

In [43]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])